In [1]:
import cv2
import numpy as np
import cupy as cp
import time
import imutils
from joblib import load
from dataset_io import *
import timeit
from tqdm import tqdm
import psutil
from datetime import datetime
from profiler import *

In [2]:
def pyramid(image, gradient_channels, binary_imgs, initial_scale, final_scale, scale):
    original_w=image.shape[1]
    curr_scale=initial_scale
    while curr_scale>final_scale/scale:
        w = int(original_w * curr_scale)
        yield imutils.resize(image, width=w), [imutils.resize(ch, width=w) for ch in gradient_channels], [imutils.resize(np.float32(img), width=w).astype(bool) for img in binary_imgs],curr_scale
        curr_scale/=scale
        
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [3]:
def area(rect):
    ((x,y),(x2,y2))=rect
    return (x2-x)*(y2-y)

def calc_overlapping(a,b,method):
    dx = min(a[1][0], b[1][0]) - max(a[0][0], b[0][0])
    dy = min(a[1][1], b[1][1]) - max(a[0][1], b[0][1])
    basic_area=method([area(a), area(b)])
    overlap_area=dx*dy
    is_overlapping=((dx>=0) and (dy>=0))
    overlapping_ratio=overlap_area/basic_area
    
    return overlapping_ratio if is_overlapping else 0

def scale_many(vals, scale):
    return [int(val/scale) for val in vals]

## Load classifier

In [4]:
os.chdir(PROJECT_PATH)
clf_grad = load('rand_forest_clf.joblib') 
clf_grad.set_params(n_jobs=-1)

color_clfs=[load('naive_bayes_clf_{}.joblib'.format(color)) for color in COLORS]
color_clf_map=dict(zip(COLORS, color_clfs))

## Set params

In [5]:
(winW, winH) = (WIN_W, WIN_H)
channels_no=8
DS_min_winH_m=41
DS_max_winH_m=287
initial_scale=winH/DS_min_winH_m
final_scale=winH/DS_max_winH_m
RESIZING_SCALE=1.15
MAX_PRED_OVERLAPPING=0.5
MIN_PRED=0.8
WHITE=(255, 255, 255)
DRAW_BORDER=2
KERNEL_W_RATIO=7.0/640.0 #because kernel (7,7) was the best for VGA images in previous research

In [6]:
def binarize(color_channels, color_clfs):
    s0, s1, s2 = color_channels[0].shape
    flat_ch = [np.reshape(img, (s0*s1, s2)) for img in color_channels]
    flat_img = np.hstack(flat_ch)
    flat_binaries=[clf.predict(flat_img) for clf in color_clfs]
    img_binaries=[np.reshape(flat, (s0, s1)) for flat in flat_binaries]
    return img_binaries

def open_bin(img, kernel):
    kernel = np.ones(kernel, np.uint8) 
    img_erosion = cv2.erode(img, kernel, iterations=1)
    img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)
    return img_dilation

def close_bin(img, kernel):
    kernel = np.ones(kernel, np.uint8) 
    img_dilation = cv2.dilate(img, kernel, iterations=1)
    img_erosion = cv2.erode(img_dilation, kernel, iterations=1)
    return img_erosion

def morpho(img):
    h,w=img.shape
    k=int(w*KERNEL_W_RATIO)
    kernel=(k,k)
    return np.bool_(close_bin(open_bin(np.uint8(img), kernel), kernel))

def color_score(window):
    h=win_H/1.2
    w=win_W/1.2
    x0=0.1*win_W
    y0=0.1*win_H
    x1=0.9*win_W
    y1=0.9*win_H
    rect=img[int(y0):int(y1),int(x0):int(x1)]
    ((hx1,hy1),(hx2,hy2))=((int(0.126*w+x0), int(0.3056*h+y0)),(int(0.409*w+x0), int(h+y0)))
    hole_1=img[hy1:hy2, hx1:hx2]
    ((hx1,hy1),(hx2,hy2))=((int(0.5906*w+x0), int(0.3056*h+y0)),(int(0.875*w+x0), int(h+y0)))
    hole2=img[hy1:hy2, hx1:hx2]
    all_points=np.multiply(rect.shape)
    hole_1_points=np.multiply(hole_1.shape)
    hole_2_points=np.multiply(hole_2.shape)
    hole_1_positives=np.sum(hole_1)
    hole_2_positives=np.sum(hole_2)
    rect_score=np.sum(rect)-hole_1_positives-hole_2_positives
    hole_1_score=hole_1_points-hole_1_positives
    hole_2_score=hole_2_points-hole_2_positives
    score=(rect_score+hole_1_score+hole_2_score)/all_points
    
    return score

In [7]:
def show(img, x, y, winW, winH, positives=[]):
    clone = np.float32(img.copy())
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), WHITE, DRAW_BORDER)
    for p in positives:
        x,y=p
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), WHITE, DRAW_BORDER)
    cv2.imshow("Window", clone)
    cv2.waitKey(1)

def pred_overlapping(row, scaled_rect):
    x,y,h,w = scaled_rect
    a = ((x,y),(x+w,y+h))
    [scene_name, filename, scale, (rx,ry)] = row
    [rx,ry,rw,rh] = scale_many([rx,ry,winW,winH], scale)
    b = ((rx,ry),(rx+rw,ry+rh))

    return calc_overlapping(a,b,np.min)
    
def image_predict(X,IDX,rects_count):
    global glob_RES
    if len(X)!=0 and rects_count>0:
        pred=np.array([p[1] for p in clf_grad.predict_proba(X)])
        greater_than_09=pred > MIN_PRED
        pred=pred[greater_than_09]
        IDX=IDX[greater_than_09]
        max_pred_ids=[]
        max_preds=[]
        max_pred_IDXs=[]
        for i in range(rects_count):
            if len(pred) >0:
                max_pred_id=np.argmax(pred)
                max_pred=pred[max_pred_id]
                max_preds.append(max_pred)
                max_pred_IDX=IDX[max_pred_id]
                max_pred_IDXs.append(max_pred_IDX)

                [scene_name, filename, scale, (x,y)]=max_pred_IDX
                scaled_rect=scale_many([x,y,winW,winH], scale)

                not_within_bools = np.array([pred_overlapping(row, scaled_rect) < MAX_PRED_OVERLAPPING for row in IDX])
                pred=pred[not_within_bools]
                IDX=IDX[not_within_bools]
            else:
                print("No more predictions with prob > 0.8")
        if len(max_preds)>0:
            max_preds=np.array([[e] for e in max_preds])
            res=np.append(max_pred_IDXs, max_preds, 1)
            glob_RES=np.vstack([glob_RES,res])
    
def predict(filename, scene_name, label_resolution, rects, *args):
    X=[]
    IDX=[]
    
    image=imread_resized(scene_name, filename, label_resolution)
    gradient_channels=read_split_channels(scene_name, filename, "channels_gradient")
    color_channels=read_split_channels(scene_name, filename, "channels_color")
    binary_imgs=binarize(color_channels, color_clfs)
    binary_imgs=[morpho(img) for img in binary_imgs]
    for resized_img, resized_gradient_ch, resized_binary_imgs, scale in pyramid(image, gradient_channels, binary_imgs, initial_scale, final_scale, scale=RESIZING_SCALE):
        resized_ch=np.dstack(resized_gradient_ch)
        for (x, y, window) in sliding_window(resized_img, stepSize=4, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            channels_window=resized_ch[y:y + winH, x:x + winW]
            color_ch_windows=[img[y:y + winH, x:x + winW] for img in resized_binary_imgs]
            color_scores=[color_score(w) for w in color_ch_windows]
            print(color_scores)
            color_ch_windows=None
            X.append(channels_window)
            row=[scene_name, filename, scale, (x,y)]
            IDX.append(row)
            #show(resized_binary_imgs[0], x, y, winW, winH)
    X=np.reshape(X,(len(X),winW*winH*channels_no)) #cupy does not help
    image_predict(X,np.array(IDX), len(rects))

In [8]:
glob_RES=np.empty((0,5), object)

glob_DATETIME = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")

def predict_all():
    [predict(*row) for row in tqdm([row for row in walk_dataset()])]# if row[1]=="warehouse_1"])]# and row[0]=="r_1_0.jpg"])]
    
profiled('predict_all()', globals(), locals())

  8%|▊         | 162/2092 [04:58<59:24,  1.85s/it]  

No more predictions with prob > 0.8


  8%|▊         | 163/2092 [05:00<1:01:18,  1.91s/it]

No more predictions with prob > 0.8


 10%|█         | 211/2092 [06:43<1:07:36,  2.16s/it]

...

 96%|█████████▋| 2015/2092 [1:08:19<02:18,  1.80s/it]

No more predictions with prob > 0.8


 96%|█████████▋| 2016/2092 [1:08:21<02:16,  1.80s/it]

No more predictions with prob > 0.8


100%|██████████| 2092/2092 [1:11:05<00:00,  2.04s/it]


In [9]:
def calc_overlappings(rects, pred_rects):
    global glob_overlappings    
    overlappings=[(np.max([calc_overlapping(pred_rect,rect, np.max) for pred_rect in pred_rects]) if len(pred_rects) > 0 else 0) for rect in rects]
    glob_overlappings.extend(overlappings)

def draw_predicted_rectangles(filename, scene_name, label_resolution, rects):
    global glob_RES
    
    img=imread_resized(scene_name, filename, label_resolution)
    img_rows=glob_RES[np.where((glob_RES[:,0] == scene_name) * (glob_RES[:,1] == filename))]
    
    pred_rects=[]
    rects=[add_margin(correct_rect_ratio(rect)) for rect in rects]
    for max_row in img_rows:
        [scene_name, filename, scale, (x,y), pred]=max_row
        [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
        cv2.rectangle(img, (x,y), (x + winW_s, y + winH_s), (0, 255, 0), 2)
        pred_rects.append(((x,y),(x+winW_s, y+winH_s)))
    for rect in rects: 
        ((x,y),(x2,y2))=rect
        cv2.rectangle(img, (x,y), (x2,y2), (255, 0, 0), 2)
    
    save_image(img, scene_name, filename, "predicted_labels")
    calc_overlappings(rects, pred_rects)

glob_overlappings=[]
[draw_predicted_rectangles(*row) for row in tqdm(walk_dataset())]# if row[1]=="warehouse_1"]# and row[0]=="r_1_0.jpg"]
print("Mean overlapping: ",np.mean(glob_overlappings))

  0%|          | 0/2092 [00:00<?, ?it/s]

TypeError: draw_predicted_rectangles() takes 4 positional arguments but 5 were given

In [ ]:
#min_pred=0.8 : overlap = 0.623